## Imports

In [1]:
import os

## Generate Slice Data

In [2]:
DATA_DIR = os.path.join("..","..","data")
ORIGINAL_DATA_PATH = os.path.join(DATA_DIR,"BrainTriage")
SLICED_DATA_DIR = os.path.join(DATA_DIR, "sliced-data")
TRAIN_PATH = os.path.join(SLICED_DATA_DIR, "train")
TEST_PATH = os.path.join(SLICED_DATA_DIR, "test")
DATA_SPLIT_DIR = os.path.join(DATA_DIR, "data-split")
TMP_DIR = "tmp"
print(DATA_DIR, ORIGINAL_DATA_PATH, SLICED_DATA_DIR, TRAIN_PATH, TEST_PATH, DATA_SPLIT_DIR, sep="\n")

../../data
../../data/BrainTriage
../../data/sliced-data
../../data/sliced-data/train
../../data/sliced-data/test
../../data/data-split


In [ ]:
if not os.path.exists(TRAIN_PATH):
    !python3 dataset/create_slices.py -d $ORIGINAL_DATA_PATH -o $SLICED_DATA_DIR --train
if not os.path.exists(TEST_PATH):
    !python3 dataset/create_slices.py -d $ORIGINAL_DATA_PATH -o $SLICED_DATA_DIR --test

## Create Train/Validation Split

In [ ]:
K = 10

In [ ]:
if not os.path.exists(DATA_SPLIT_DIR):
    !python3 dataset/create_data_split.py -k $K -d $TRAIN_PATH -ds $DATA_SPLIT_DIR

## Train ResNet34

In [ ]:
CNN_NAME = RESNET_TYPE = "resnet34"
SEED = 420
LR = 0.0001
EPOCHS = 30
BATCH_SIZE = 16
N_FEATURES = 128
FLIP_PROBABILITY = ROTATE_PROBABILITY = 0.5
MODEL_DIR = "../../models"

In [ ]:
!python3 train/train-resnet.py $CNN_NAME $RESNET_TYPE -s $SEED -lr $LR -e $EPOCHS -b $BATCH_SIZE  \
                               -m $MODEL_DIR -f $N_FEATURES -d $TRAIN_PATH -ds $DATA_SPLIT_DIR \
                               -afp $FLIP_PROBABILITY -arp ROTATE_PROBABILITY -ts 0 32 --tuple 

## Train LSTM (Freeze ResNet34)

In [ ]:
with open(os.path.join(TMP_DIR, CNN_NAME), 'r') as f:
    CNN_LOC = f.read()
LSTM_NAME = "lstm"
BATCH_SIZE = 2

In [ ]:
!python3 train/train-lstm.py $LSTM_NAME $RESNET_TYPE -s $SEED -d $TRAIN_PATH -ds $DATA_SPLIT_DIR -c $CNN_LOC \
                             -lr $LR -e $EPOCHS -b $BATCH_SIZE -m $MODEL_DIR -f $N_FEATURES \
                             -afp $FLIP_PROBABILITY -arp $ROTATE_PROBABILITY -ts 0 32 --tuple

## Finetune CombinedNet

In [ ]:
with open(os.path.join(TMP_DIR, LSTM_NAME), 'r') as f:
    LSTM_LOC = f.read()
COMBINED_NAME = "combinednet"

In [ ]:
!python3 train/train-combinednet.py $COMBINED_NAME $RESNET_TYPE -s $SEED -d TRAIN_PATH -ds $DATA_SPLIT_DIR \
                                    -l LSTM_LOC -lr $LR -e $EPOCHS -b $BATCH_SIZE -m $MODEL_DIR -f $N_FEATURES \
                                    -afp $FLIP_PROBABILTY -arp $ROTATE_PROBABILITY -ts 0 32 --tuple

## Clean-up

In [ ]:
try:
    shutil.rmtree(TMP_DIR)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))